In [1]:
!pip3 install pyspark

In [2]:
from pyspark import SparkContext

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Spark master") \
    .getOrCreate()
sc = spark.sparkContext


In [2]:
spark

### Resilient Distributed Dataset or RDD

An RDD is a distributed collection of elements. All work in Spark is expressed as either creating new RDDs, transforming existing RDDs, or calling actions on RDDs to compute a result. Spark automatically distributes the data contained in RDDs across your cluster and parallelizes the operations you perform on them.

https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=flatmap#pyspark.RDD.cache

#### Creating and RDD using parallelize
Another way of creating an RDD is to parallelize an already existing list.

In [23]:
A = ((a, a*a) for a in range(1000))

In [24]:
data = sc.parallelize(A)

In [25]:
data.count()

1000

In [26]:
data.take(10)

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

#### Creating a RDD from a file
The most common way of creating an RDD is to load it from a file. Notice that Spark's textFile can handle compressed files directly.

In [3]:
file = sc.textFile('konstytucja.txt')

In [4]:
file.count()

808

In [6]:
file.take(10)

['Konstytucja Rzeczypospolitej Polskiej/Rozdział 1',
 '',
 '',
 '',
 '',
 '',
 'Pobrano z Wikiźródeł dnia 02/14/20',
 '',
 '',
 '']

In [7]:
filtered = file.filter(lambda x: x != '')

In [8]:
filtered.take(10)

['Konstytucja Rzeczypospolitej Polskiej/Rozdział 1',
 'Pobrano z Wikiźródeł dnia 02/14/20',
 'Konstytucja Rzeczypospolitej Polskiej',
 'Rozdział 1',
 'Konstytucja Rzeczypospolitej Polskiej',
 'Rozdział 1 Rzeczpospolita — Rozdział 2 Wolności, prawa i obowiązki człowieka i obywatela, zasady ogólne wolności, prawa osobiste, wolności i prawa polityczne, wolności i prawa ekonomiczne, socjalne i kulturalne, środki ochrony wolności i praw, obowiązki — Rozdział 3 Źródła prawa — Rozdział 4 Sejm i Senat, wybory i kadencja, posłowie i senatorowie, organizacja i działanie, referendum — Rozdział 5 Prezydent Rzeczypospolitej Polskiej — Rozdział 6 Rada Ministrów i administracja rządowa — Rozdział 7 Samorząd terytorialny — Rozdział 8 Sądy i Trybunały, Trybunał Konstytucyjny, Trybunał Stanu — Rozdział 9 Organy kontroli państwowej i ochrony prawa, Najwyższa Izba Kontroli, Rzecznik Praw Obywatelskich, Krajowa Rada Radiofonii i Telewizji — Rozdział 10 Finanse publiczne — Rozdział 11 Stany nadzwyczajne — R

In [16]:
counts = filtered \
    .flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda acc, item: acc + item)


In [18]:
counts.takeOrdered(5)

[('(Dz.', 2),
 ('(dobra', 1),
 ('(dostęp', 1),
 ('(godność', 1),
 ('(humanitarne', 1)]

In [21]:
counts.takeOrdered(5, key=lambda x: -x[1])

[('i', 173), ('w', 127), ('Rozdział', 75), ('—', 60), ('Art.', 57)]